# Instituto Tecnológico de Costa Rica

### Escuela de Ingeniería en Computación

### Maestría Académica en Ciencias de la Computación

### Curso: Aprendizaje Automático (Machine Learning)

# Proyecto Final
- Agregar nombre del proyecto
## Estudiantes:
- Emmanuel Rosales Salas - 2013108931
- Alejandro Arroyo Valle

# Carga de Bibliotecas y Dataset

In [32]:
# Instalar las bibliotecas necesarias
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Importación de la clases
import pandas as pd
import numpy as np

In [34]:
# Importar los datos.
raw_dataset = pd.read_csv('multi_data_01.csv', header = 0, parse_dates = [0], index_col = 0)
raw_dataset.head()

,runautomode_bool,actual_speed_m_min,secundary_pumpspeed_rpm,secundary_pa16d1_bar,secundary_p014d1_bar,secundary_p015d1_bar,secundary_pa7b3_mm,secundary_pa4b2_c,secundary_pa4b5_c,secundary_pu2b2_c,...,primary_p015d1_bar,primary_pa7b3_mm,primary_pa4b2_c,primary_pa4b5_c,primary_pu2b2_c,primary_p012b3_c,primary_p022b3_c,primary_pa16d6_c,primary_p014d6_c,primary_p015d6_c
date_time,,,,,,,,,,,,,,,,,,,,,
2023-06-22 14:17:24.698380,t,3.500000,21.831861,33.174477,94.676770,57.584450,33.328247,99.9,103.1,99.9,...,76.056860,225.26270,101.3,99.7,99.9,86.3,100.2,869.9,98.9,103.5
2023-06-22 14:17:30.468259,t,3.702725,25.683084,32.924850,97.399980,59.626854,33.132996,99.9,103.1,99.9,...,81.423840,224.77570,101.7,99.7,99.8,86.1,100.1,869.9,98.9,103.3
2023-06-22 14:17:35.891179,t,4.082303,28.471409,32.860012,102.812355,63.836475,33.035340,99.9,103.2,99.9,...,81.900406,224.58087,101.7,99.7,99.6,86.1,100.2,869.9,98.8,103.2
2023-06-22 14:17:40.972308,t,4.322869,30.506277,32.804900,106.148280,66.593720,33.100403,99.9,103.2,99.9,...,81.696170,224.45099,101.9,99.6,99.7,85.8,100.0,869.9,98.7,103.1
2023-06-22 14:17:46.051777,t,4.709167,32.320232,32.584450,111.163510,70.587750,33.035340,99.9,103.2,99.9,...,86.416390,224.54840,101.9,99.6,99.7,85.6,99.9,869.9,98.6,102.9


# Exploración de Datos

In [35]:
print("Descripción del Dataset");
print(raw_dataset.describe());

Descripción del Dataset
       actual_speed_m_min  secundary_pumpspeed_rpm  secundary_pa16d1_bar  \
count       939781.000000            939781.000000         939781.000000   
mean             3.173700                35.535232             25.243818   
std              3.198930                29.190585             13.188008   
min              0.000000                 0.000000            -24.998379   
25%              0.000000                 0.110744             27.228813   
50%              3.000000                40.756966             30.691174   
75%              6.500000                56.406920             34.030342   
max              7.899999               100.000000             42.067040   

       secundary_p014d1_bar  secundary_p015d1_bar  secundary_pa7b3_mm  \
count         939781.000000         939781.000000       939781.000000   
mean              76.610402             47.184574          312.270014   
std               60.827016             43.669193          164.502212   

In [36]:
print(f"Las dimensiones del dataset son (filas, columnas): {raw_dataset.shape}")

Las dimensiones del dataset son (filas, columnas): (939781, 29)


# Preprocesamiento de Datos

### Eliminación de las columnas primary

[Agregar justificación]

In [37]:
# Eliminar las columnas relacionadas con primary 

# Primero se obtienen los nombres de las columnas que contienen 'primary'
primary_columns = raw_dataset.filter(like='primary').columns

# Eliminamos las columnas del DataFrame
raw_dataset = raw_dataset.drop(columns = primary_columns)

# Imprimir la cantidad de columnas eliminadas y las primeras filas del nuevo DataFrame
print("\nDataFrame después de eliminar las columnas:")
print(f"Se eliminaron {len(primary_columns)}")
raw_dataset.head()



DataFrame después de eliminar las columnas:
Se eliminaron 13


,runautomode_bool,actual_speed_m_min,secundary_pumpspeed_rpm,secundary_pa16d1_bar,secundary_p014d1_bar,secundary_p015d1_bar,secundary_pa7b3_mm,secundary_pa4b2_c,secundary_pa4b5_c,secundary_pu2b2_c,secundary_p012b3_c,secundary_p022b3_c,secundary_pa16d6_c,secundary_p014d6_c,secundary_p015d6_c,secundary_multipressure_bar
date_time,,,,,,,,,,,,,,,,
2023-06-22 14:17:24.698380,t,3.500000,21.831861,33.174477,94.676770,57.584450,33.328247,99.9,103.1,99.9,93.9,99.9,102.2,95.8,108.8,15.024406
2023-06-22 14:17:30.468259,t,3.702725,25.683084,32.924850,97.399980,59.626854,33.132996,99.9,103.1,99.9,93.9,99.8,102.1,95.8,108.8,14.090146
2023-06-22 14:17:35.891179,t,4.082303,28.471409,32.860012,102.812355,63.836475,33.035340,99.9,103.2,99.9,93.8,100.1,102.2,95.8,108.6,14.032947
2023-06-22 14:17:40.972308,t,4.322869,30.506277,32.804900,106.148280,66.593720,33.100403,99.9,103.2,99.9,93.8,100.3,102.2,95.8,108.6,13.346553
2023-06-22 14:17:46.051777,t,4.709167,32.320232,32.584450,111.163510,70.587750,33.035340,99.9,103.2,99.9,93.8,100.3,102.2,95.8,108.6,13.003356


# Modelo de Series de tiempo

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

# Resultados